In [1]:
from rabbit_clients import publish_message, consume_message
import pandas_profiling
import pandas as pd
import datetime
import re
import numpy as np
import os

In [24]:
os.getcwd()

'/home/jovyan/work'

In [4]:
@publish_message(queue='test')
def publish_greeting():
    message={'greeting':'yo'}
    return message

#@consume_message(consume_queue='test')
#def consume_greeting():
#    publish_greeting()
#    return body

if __name__ == '__main__':
    publish_greeting()


AMQPConnectionError: 

In [14]:
print(body)

NameError: name 'body' is not defined

In [4]:
conmessage={
        "fileid" : "1234",
        "filename" : 'iris.csv',
        "username" : 'PConsuela',
        "datetime" : datetime.datetime.now(),
        "fileloc" : "/home/jovyan/work"
}

In [6]:
def get_filelocs(message_dict):
    fileloc=message_dict['fileloc']+'/'+message_dict['filename']
    infiletype=message_dict['filename'].split(".",1)[1]
    return_filename=message_dict['filename'].split(".",1)[0]+"_profile.html"
    return fileloc, infiletype, return_filename

In [10]:
def send_error_message(err, message_dict):
    message = {
        "fileid": message_dict["fileid"],
        "filename": message_dict["filename"],
        "username": message_dict["username"],
        "datetime": str(datetime.datetime.now()),
        "fileloc": message_dict["fileloc"],
        "message": err
        }
    return message

In [12]:
def send_complete_message(message_dict):
    return_message = {
        "fileid": message_dict["fileid"]+"_profile",
        "filename": return_filename,
        "username": message_dict["username"],
        "datetime": str(datetime.datetime.now()),
        "fileloc": message_dict["fileloc"]
    }
    return return_message

In [14]:
def read_file(fileloc, infiletype, message_dict):
    data=[]
    try:
        if infiletype == 'txt':
            with open(fileloc, 'r') as f:
                data.append(f.read())
        elif infiletype in ('xls', 'xlsx'):
            data = pd.read_excel(fileloc)
        else:
            data = pd.read_csv(fileloc)
        df=pd.DataFrame(data)
    except FileNotFoundError:
        df=None
#        @publish_message(queue='error')
        send_error_message("File does not exist", message_dict)
    except pd.errors.ParserError:
        df=None
#        @publish_message(queue='error')
        send_error_message("Unacceptable file type.  Accepted file types are .csv, .txt, .xls and .xlsx", message_dict)
    return df

In [16]:
def profile(df, message_dict):
    try:
        profile=df.profile_report()
    except AttributeError:
        profile=None
#        @publish_message(queue='error')
        send_error_message("Profiler could not complete", message_dict)
    return profile

In [19]:
def write_file(profile, return_filename, message_dict):
    try:
        profile.to_file(output_file=return_filename)  
        #@publish_message(queue='file ready')
        send_complete_message(message_dict, return_filename)
    except:
        #@publish_message(queue='error')
        send_error_message("Profile could not be written", message_dict)

In [ ]:
if __name__ == '__main__':
    #@consume_message(consume_queue='profiler')
    filelocs = get_filelocs(conmessage)
    file = read_file(filelocs[0], filelocs[1], conmessage)
    profile = profile(file, conmessage)
    write_file(profile, filelocs[3])